# Project Title: Predictive Trading Strategy Development for VDE ETF Using Machine Learning- Part-2

# By Abhishek Patil



# Here were taking the input which is our delta:

In [1]:
delta = input("Hello! How much money would you like to make today(in percent %): ")
delta = float(delta)
delta = delta / 100
#print(delta)

Hello! How much money would you like to make today(in percent %): 0.5


# Loading all the necessary Libraries:

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from datetime import date, timedelta
import pandas as pd

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


# Running all the Machine learning Models:

In [3]:
# Returns the current local date
today = pd.to_datetime(date.today())


# Taking the weekday of the week
weekday = today.weekday()


#Filtering for yesterday for a 5 day week
if weekday in [1,2,3,4,5]:
  yesterday = today - timedelta(days=1)
elif weekday == 0:
  yesterday = today - timedelta(days=3)
elif weekday == 6:
  yesterday = today - timedelta(days=2)



# Download the Data
ticker = "VDE"
vde = yf.download(ticker, start='2000-01-01', end=today)

vde['Daily_Return'] = vde['Close'].pct_change()
vde.dropna(inplace=True)
vde["Move"] = vde["High"] - vde["Low"]
vde["Move_per"] = vde["Move"] / vde["Low"]
vde['Target'] = np.where(vde['Daily_Return'] > 0, 1, 0)







#Random Forest Classifier
# Define features and target
features = ["Open", "Close", "High", "Low","Volume","Daily_Return", "Move", "Move_per"]
X = vde[features]
y = vde['Target']

#List of training end dates
train_end_date = yesterday
# train_end_date = ts[0]
# print(train_end_date)

# Train-test split
X_train = X[X.index < train_end_date]
y_train = y[y.index < train_end_date]
X_test = X[X.index >= train_end_date]
y_test = y[y.index >= train_end_date]

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
predictions = model.predict(X_test_scaled)
print(predictions)

#Giving out the Signal
if predictions == 1:
  print(f"Today {today}:\n\n Long position.\n\n")
elif predictions == 0:
  print(f"Today {today}:\n\n Short or Neutral position.\n\n")



#Multiple Regression
features_open = ["Close", "High", "Low","Volume","Daily_Return", "Move", "Move_per"]
features_high =["Close", "Open", "Low","Volume","Daily_Return", "Move", "Move_per"]
features_low = ["Close", "Open", "High","Volume","Daily_Return", "Move", "Move_per"]
features_close = ["Open", "High", "Low","Volume","Daily_Return", "Move", "Move_per"]
features_Daily_Return =["Close", "Open", "High","Volume","Low", "Move", "Move_per"]
features_Move_per = ["Close", "Open", "High","Volume","Low", "Move", "Daily_Return"]

features = [features_open, features_high, features_low, features_close, features_Daily_Return, features_Move_per]
targets = ["Open", "High", "Low", "Close", "Daily_Return", "Move_per"]

# Create an empty dictionary to store todays prediction
todays_predictions = {}

#Create an for loop to loop over features and target to predict the values of the next day
for i,x in zip(features,targets):
  X = vde[i]
  y = vde[x]

  #List of training end dates
  train_end_date = yesterday
  # train_end_date = ts[0]
  # print(train_end_date)

  # Train-test split
  X_train = X[X.index < train_end_date]
  y_train = y[y.index < train_end_date]
  X_test = X[X.index >= train_end_date]
  y_test = y[y.index >= train_end_date]

  # Standardize features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)


  # Multiple Regression
  lr_model = LinearRegression()
  lr_model.fit(X_train_scaled, y_train)
  lr_predictions = lr_model.predict(X_test_scaled)

  # Store predictions in the dictionary
  todays_predictions[x] = lr_predictions


# Convert the dictionary to a DataFrame
predictions_df = pd.DataFrame(todays_predictions)


df = vde[(vde["Daily_Return"] > 0) & (vde["Move_per"] >= delta)]
count_whole = len(vde)
count = len(df)

prob = round((count/count_whole)*100,2)
print(f"Prediction of Multiple Linear Regression Model:\n\nThe probability of making {delta} on {today} is {prob}% with these predictions:\n\n {predictions_df}\n\n")





#Random Forest
features_open = ["Close", "High", "Low","Volume","Daily_Return", "Move", "Move_per"]
features_high =["Close", "Open", "Low","Volume","Daily_Return", "Move", "Move_per"]
features_low = ["Close", "Open", "High","Volume","Daily_Return", "Move", "Move_per"]
features_close = ["Open", "High", "Low","Volume","Daily_Return", "Move", "Move_per"]
features_Daily_Return =["Close", "Open", "High","Volume","Low", "Move", "Move_per"]
features_Move_per = ["Close", "Open", "High","Volume","Low", "Move", "Daily_Return"]

features = [features_open, features_high, features_low, features_close, features_Daily_Return, features_Move_per]
targets = ["Open", "High", "Low", "Close", "Daily_Return", "Move_per"]

# Create an empty dictionary to store todays prediction
todays_predictions = {}

#Create an for loop to loop over features and target to predict the values of the next day
for i,x in zip(features,targets):
  X = vde[i]
  y = vde[x]

  #List of training end dates
  train_end_date = yesterday
  # train_end_date = ts[0]
  # print(train_end_date)

  # Train-test split
  X_train = X[X.index < train_end_date]
  y_train = y[y.index < train_end_date]
  X_test = X[X.index >= train_end_date]
  y_test = y[y.index >= train_end_date]

  # Standardize features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)


  # Random Forest Regression
  rf_model = RandomForestRegressor()
  rf_model.fit(X_train_scaled, y_train)
  rf_predictions = rf_model.predict(X_test_scaled)


  # Store predictions in the dictionary
  todays_predictions[x] = rf_predictions


# Convert the dictionary to a DataFrame
predictions_df = pd.DataFrame(todays_predictions)


df = vde[(vde["Daily_Return"] > 0) & (vde["Move_per"] >= delta)]
count_whole = len(vde)
count = len(df)

prob = round((count/count_whole)*100,2)
print(f"Prediction of Random Forest Model:\n\nThe probability of making {delta} on {today} is {prob}% with these predictions:\n\n {predictions_df}\n\n")

[*********************100%%**********************]  1 of 1 completed


[0]
Today 2024-01-30 00:00:00:

 Short or Neutral position.


Prediction of Multiple Linear Regression Model:

The probability of making 0.005 on 2024-01-30 00:00:00 is 51.26% with these predictions:

         Open        High         Low       Close  Daily_Return  Move_per
0  116.82284  117.610001  116.279999  116.858046      0.005043  0.009162


Prediction of Random Forest Model:

The probability of making 0.005 on 2024-01-30 00:00:00 is 51.26% with these predictions:

        Open        High       Low       Close  Daily_Return  Move_per
0  117.0315  117.955501  116.7065  116.992199      -0.00007  0.011338


